# [Finnhub](https://finnhub.io/docs/api/company-news)
Limitations:
* 60 calls/minute
* max 1 year of historical data
* US companies only (no crypto etc.)

In [1]:
#arguments

fh_key = 'cqnr07hr01qo8864qbpgcqnr07hr01qo8864qbq0'
fh_tickers = ['AAPL', 'META'] #TODO select stocks
fh_from = '2023-01-01' #YYYY-MM-DD
fh_to = '2025-01-01'

In [2]:
#collection

import finnhub
import pandas as pd
from time import sleep

fh_client = finnhub.Client(fh_key)
fh_df = pd.DataFrame()
for fh_ticker in fh_tickers:
    print('\n' + fh_ticker, end='')

    fh_data = fh_client.company_news(fh_ticker, fh_from, fh_to)
    fh_df = pd.concat([fh_df, pd.DataFrame(fh_data)], ignore_index=True)
    while True:
        sleep(1)
        fh_last = pd.Timestamp(fh_df['datetime'].iat[-1] - 86400, unit='s').strftime('%Y-%m-%d') #day before
        fh_data = fh_client.company_news(fh_ticker, fh_from, fh_last)
        if not fh_data:
            break
        fh_df = pd.concat([fh_df, pd.DataFrame(fh_data)], ignore_index=True)
        print('.', end='')

fh_df['datetime'] = pd.to_datetime(fh_df['datetime'], unit='s')


AAPL...................................................................
META...............................................

In [3]:
fh_df

,category,datetime,headline,id,image,related,source,summary,url
0,company,2024-09-09 06:33:24,Apple iPhone 16 Event: What to Expect,129782414,,AAPL,Yahoo,"In a few hours, Apple will unveil its newest i...",https://finnhub.io/api/news?id=85d752fb8f9c069...
1,company,2024-09-09 06:12:09,Apple Considers Expanding India Production Fur...,129782415,https://media.zenfs.com/en/Benzinga/52406c1efd...,AAPL,Yahoo,This story was first published on the Benzinga...,https://finnhub.io/api/news?id=c47c669886529ed...
2,company,2024-09-09 02:58:17,Dow Jones Futures Rise: Apple iPhone 16 On Tap...,129780927,https://media.zenfs.com/en/ibd.com/2faedde23bd...,AAPL,Yahoo,Monday's Apple iPhone 16 unveiling is key for ...,https://finnhub.io/api/news?id=b54454b7420879e...
3,company,2024-09-09 02:38:35,Watch These Apple Stock Price Levels as iPhone...,129781409,https://www.investopedia.com/thmb/rvpYmQzE9r0U...,AAPL,Yahoo,Apple Shares Have Rallied About 13% From Their...,https://finnhub.io/api/news?id=c8bf68dd001087d...
4,company,2024-09-09 01:32:45,Apple: Not Surprising That Berkshire Sold Half,129781733,https://static.seekingalpha.com/cdn/s3/uploads...,AAPL,SeekingAlpha,Apple's revenue and cash flow growth haven't b...,https://finnhub.io/api/news?id=cea67989477537a...
...,...,...,...,...,...,...,...,...,...
25434,company,2023-09-15 04:00:08,WhatsApp explores ads in chat app as Meta seek...,122602269,,META,Yahoo,Discussions are at an early stage and include ...,https://finnhub.io/api/news?id=b3c771487c2a7bc...
25435,company,2023-09-15 03:33:00,Meta Platforms Inc. stock underperforms Friday...,122616754,,META,MarketWatch,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=1e248480c72e5c7...
25436,company,2023-09-15 03:28:00,What You Missed On Wall Street On Friday,122614738,,META,Thefly.com,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=0c6f2946efa416e...
25437,company,2023-09-15 01:07:00,"For better gains in tech stocks from here, loo...",122613495,,META,MarketWatch,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=9e29fb45837f95e...


# [Alpha Vantage](https://www.alphavantage.co/documentation/#news-sentiment)
Limitations:
* 25 calls/day
* max 1000 results/call

In [2]:
#arguments

av_key = 'O9R21IG9ZRS8AEAK'
av_tickers = ['CRYPTO:BTC', 'CRYPTO:SOL'] #TODO select cryptos
av_from = '20240801T0000' #YYYYMMDDTHHMM
av_to = '' #optional
av_limit = 1000

In [4]:
#TODO double check code below
#TODO what if reaches limit?

In [3]:
#collection

import requests

av_df = pd.DataFrame()
for av_ticker in av_tickers:
    print('\n' + av_ticker, end='')

    av_url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&apikey={av_key}&tickers={av_ticker}&time_from={av_from}&time_to={av_to}&limit={av_limit}'
    av_data = [{**d, 'ticker': av_ticker.split(':')[-1]} for d in requests.get(av_url).json()['feed']]
    av_df = pd.concat([av_df, pd.DataFrame(av_data)], ignore_index=True)
    while True:
        av_last = av_df['time_published'].iat[-1][:-2] #trim seconds
        av_url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&apikey={av_key}&tickers={av_ticker}&time_from={av_from}&time_to={av_last}&limit={av_limit}'
        av_data = [{**d, 'ticker': av_ticker.split(':')[-1]} for d in requests.get(av_url).json()['feed']][1:] #skip first
        if not av_data:
            break
        av_df = pd.concat([av_df, pd.DataFrame(av_data)], ignore_index=True)
        print('.', end='')


CRYPTO:BTC..
CRYPTO:SOL

In [12]:
av_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker
0,Nasdaq On Track To Lead Market Rebound: What's...,https://www.benzinga.com/news/earnings/24/09/4...,20240909T110104,[Shanthi Rexaline],U.S. stocks appear on track to start the new w...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.051126,Neutral,"[{'ticker': 'NVDA', 'relevance_score': '0.0805...",BTC
1,"The Ultimate Growth Stock to Buy With $1,000 R...",https://www.fool.com/investing/2024/09/09/the-...,20240909T104500,[RJ Fulton],This type of growth is hard to come by in toda...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.301163,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.0658...",BTC
2,Bitcoin Climbs 2% While Analysts Remain Wary o...,https://decrypt.co/248468/bitcoin-climbs-2-whi...,20240909T102833,[Murtuza Merchant],"""This year's Bitcoin price structure is starti...",https://cdn.decrypt.co/resize/1024/height/512/...,Decrypt.co,n/a,decrypt.co,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.031241,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",BTC
3,Bitfarms Reschedules Special Meeting of Shareh...,https://www.globenewswire.com/news-release/202...,20240909T101500,[Bitfarms Ltd.],- Provides Update on Engagement with Riot -- S...,https://ml.globenewswire.com/Resource/Download...,GlobeNewswire,n/a,www.globenewswire.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.205729,Somewhat-Bullish,"[{'ticker': 'BITF', 'relevance_score': '0.6310...",BTC
4,Bitfarms Reschedules Special Meeting of Shareh...,https://www.benzinga.com/pressreleases/24/09/g...,20240909T101500,[Globe Newswire],- Provides Update on Engagement with Riot - - ...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,News,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.203028,Somewhat-Bullish,"[{'ticker': 'BITF', 'relevance_score': '0.6158...",BTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,"Pepe Leads Rally, Dogecoin, Shiba Inu Advance ...",https://www.benzinga.com/markets/cryptocurrenc...,20240809T044225,[Aniket Verma],Memecoins were back in the green Thursday as t...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.222140,Somewhat-Bullish,"[{'ticker': 'CRYPTO:ETH', 'relevance_score': '...",SOL
1960,Trump Jr. And Eric Caution Crypto Community On...,https://www.benzinga.com/markets/cryptocurrenc...,20240809T013144,[Aniket Verma],"Donald Trump Jr. and Eric Trump, sons of forme...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.237095,Somewhat-Bullish,"[{'ticker': 'CRYPTO:SOL', 'relevance_score': '...",SOL
1961,Solana Outperforms Bitcoin Over The Past Month...,https://www.benzinga.com/markets/cryptocurrenc...,20240807T172001,[Khyathi Dalal],Solana's SOL/USD strength against Bitcoin BTC/...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.116008,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",SOL
1962,Cats In A Dogs World Beats Memecoin Puppies Do...,https://www.benzinga.com/markets/cryptocurrenc...,20240807T042727,[Aniket Verma],"Cat-themed meme coin, cat in a dogs world MEW/...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.132986,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",SOL
